In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lab 3: Digital imagery and image processing


<table align="left">
 <td>
   <a href=https://colab.research.google.com/github/KMarkert/ee-workshop-esa2023/blob/main/notebooks/03_digital_imagery_and_image_processing.ipynb>
       <img src=https://cloud.google.com/ml-engine/images/colab-logo-32px.png alt="Colab logo">
    Run in Colab
   </a>
 </td>
 <td>
   <a href=https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/KMarkert/ee-workshop-esa2023/main/notebooks/03_digital_imagery_and_image_processing.ipynb>
       <img src=https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32 alt=\"Vertex AI logo\">
     Open in Vertex AI Workbench
   </a>
 </td>
</table>
<br/><br/><br/>

**Purpose:** The purpose of this lab is to demonstrate concepts of digital image processing.  You will be introduced to methods for image smoothing, sharpening, edge detection, morphological processing, texture analysis, resampling and reprojection.  At the completion of the lab, you will be able to identify image processing operators that may be useful in extracting information of interest for your image analyses.

In [ ]:
# import ee api and geemap package
import ee
import geemap
from pprint import pprint
from IPython.display import JSON

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Digital image visualization

You've learned about how an image stores pixel data in each band as DNs and how the pixels are organized spatially.  When you add an image to the map, Earth Engine handles the spatial display for you by recognizing the projection and putting all the pixels in the right place.  However, you must specify how to stretch the DNs to make an 8-bit display image (e.g. the min and max visualization parameters).  Specifying min and max applies (where $DN'$ is the displayed value):

$DN' = (DN - min) * 255 / (max - min)$

This visualization process is linear, we can apply transformations on the displayed values to highlight specific features of the image or values.

In [ ]:
# load in a Landsat 8 image directly 
# this is the image we will be using for processing
image = (
    ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')
    .select("B[1-7]")
)

Some of the image processing we are going to explore requires us to "force" Earth Engine to process at a specific projection and scale so we are going to set the image projection to a variable for later use.

In [ ]:
# extract out the projection information
proj = image.projection()

### Gamma correction

The Gamma correction is a nonlinear operation used to scale the DN values for visualization. The gamma correction can be applied mathematically ($DN' = DN^\gamma$), however, we can apply it simply providing the `gamma` keyword for visualization:

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

# Display gamma stretches of the input image.
Map.addLayer(image.visualize(bands=["B7","B5","B3"], min=0, max=5500, gamma=0.5), {}, 'gamma = 0.5');
Map.addLayer(image.visualize(bands=["B7","B5","B3"], min=0, max=5500, gamma=1.5), {}, 'gamma = 1.5');

Map

Note that gamma is supplied as an argument to `image.visualize()` so that you can click on the map to see the difference in pixel values (try it!).  It's possible to specify gamma, min and max to achieve other unique visualizations.


### Histogram equalization

Histogram equalization is a method in image processing of contrast adjustment using the image's histogram.

To apply a histogram equalization stretch, use the `sldStyle()` method:


In [ ]:
# Define a RasterSymbolizer element with '_enhance_' for a placeholder.
histogram_sld = """
  <RasterSymbolizer>
    <ContrastEnhancement><Histogram/></ContrastEnhancement>
    <ChannelSelection>
      <RedChannel>
        <SourceChannelName>B7</SourceChannelName>
      </RedChannel>
      <GreenChannel>
        <SourceChannelName>B5</SourceChannelName>
      </GreenChannel>
      <BlueChannel>
        <SourceChannelName>B3</SourceChannelName>
      </BlueChannel>
    </ChannelSelection>
  </RasterSymbolizer>
"""


In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

# Display visualization stretches of the input image.
Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(image.sldStyle(histogram_sld), {}, 'Equalized');


Map

## Band math

Band math can be performed using operators like `add()` and `subtract()`, but for complex computations with more than a couple of terms, the `expression()` function provides a good alternative.

### Normalized Difference Vegetation Index

As a simple example for using band math is calculating the Normalized Difference Vegetation Index (NDVI) using Landsat imagery, where `add()`, `subtract()`, and `divide()` operators are used:

$NDVI = \frac{NIR-Red}{NIR + Red}$

In [ ]:
nir = image.select("B5")
red = image.select("B4")

ndvi = nir.subtract(red).divide(nir.add(red))

In [ ]:
# Display original and NDVI images.
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original')
Map.addLayer(ndvi, {"min":0,"max":1}, 'NDVI');

Map

For the complete list of mathematical operators handling basic arithmetic, trigonometry, exponentiation, rounding, casting, bitwise operations and more, see the [API documentation](https://developers.google.com/earth-engine/apidocs).

### Expression example

To implement more complex mathematical expressions, consider using `image.expression()`, which parses a text representation of a math operation. The following example uses `expression()` to compute the [Enhanced Vegetation Index (EVI)](https://en.wikipedia.org/wiki/Enhanced_vegetation_index):

In [ ]:
evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': image.select('B5'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
})

In [ ]:
# Display original and NDVI images.
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original')
Map.addLayer(ndvi, {"min":0,"max":1}, 'NDVI')
Map.addLayer(evi, {"min":0,"max":2}, 'EVI')

Map

The first argument to `expression()` is the textual representation of the math operation, the second argument is a dictionary where the keys are variable names used in the expression and the values are the image bands to which the variables should be mapped. Bands in the image may be referred to as `b("band name")` or `b(index)`, for example `b(0)`, instead of providing the dictionary. Bands can be defined from images other than the input when using the band map dictionary

## Zonal statistics

To get statistics of pixel values in a region of an ee.Image, use `image.reduceRegion()`. This reduces all the pixels in the region(s) to a statistic or other compact representation of the pixel data in the region (e.g. histogram). The region is represented as a Geometry, which might be a polygon, containing many pixels, or it might be a single point, in which case there will only be one pixel in the region.



In [ ]:
# create a combined reducer that will calculate mean and standard deviation 
my_reducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True)

# calculate mean and stdDev over entire image
image_stats = image.reduceRegion(
    reducer = my_reducer,
    geometry = image.geometry(),
    scale = 120,
    maxPixels = 1e10,
    bestEffort = False
)

Note here that the reducers are combined as it is more efficient to combine reducers if you need multiple statistics (e.g. mean and standard deviation) from a single input (e.g. an image region). ([Combining reducers best practice](https://developers.google.com/earth-engine/guides/best_practices#combine-reducers))

In [ ]:
# print the statistics
JSON(image_stats.getInfo())

In [ ]:
# convert the dictionary to an image
# band names will be the keys
# bands will be constant values
statistics_image = image_stats.toImage()

In [ ]:
# extract out the mean and standard deviation bands
# to two seperate images
mean_image = statistics_image.select("B.*_mean")
stdDev_image = statistics_image.select("B.*_stdDev")

# calculate z-score
zscore = image.select("B.*").subtract(mean_image).divide(stdDev_image)

In [ ]:
# Display original and scaled images.
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original')
Map.addLayer(zscore, {"bands": ["B7","B5","B3"],"min":-2.5,"max":2.5}, 'Std. Deviation Stretch')

Map

## Neighborhood operations


## Image Thresholding

In [ ]:
# alternate approach for calculating a normalized difference
mndwi = image.normalizedDifference(["B3","B6"])

In [ ]:
threshold = 0.1 # define a threshold to segment the image
water = mndwi.gt(threshold)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(mndwi, {"min": -0.5, "max": 1,}, 'MNDWI');
Map.addLayer(water, {"min": 0, "max": 1, "palette":["black","lightblue"]}, 'Water');

Map

## Compositing

Compositing is the process of taking multiple images and making a single representative image using some statistical reduction. There are multiple ways to achieve composites and an active research area. Really the composite apporach you use depends on the data and application.

### Mean vs Median Composite

Mean/Medain compositing is very common and this illustrates the general workflow to create such a composite.

In [ ]:
# load in an image collection and filter by space and time
l8 = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterDate('2021-01-01', '2022-01-01')
    .filterBounds(ee.Geometry.Rectangle([-124.736342, 24.521208, -66.945392, 49.382808]))
)

In [ ]:
# first step is to usually mask out poor quality observations
# this function takes a landsat image and reads the QA band 
# to determine which pixels are good or bad then
# masks (sets to nodata) poor quality pixels
def qa(img):
    # Bits 3, 4 and 5 are cloud shadow, snow and cloud, respectively.
    cloudshadow_bit_mask = 1 << 3
    snow_bit_mask = 1 << 4
    clouds_bit_mask = 1 << 5
    

    # Get the pixel QA band.
    qa = img.select('pixel_qa')

    # All flags should be set to zero, indicating clear conditions.
    cloudshadow_mask = qa.bitwiseAnd(cloudshadow_bit_mask).eq(0)
    snow_mask = qa.bitwiseAnd(snow_bit_mask).eq(0)
    cloud_mask = qa.bitwiseAnd(clouds_bit_mask).eq(0)

    # combine the masks to identify where it is clear in all cases
    mask = cloudshadow_mask.And(snow_mask).And(cloud_mask)

    # Return the masked image, scaled to reflectance, without the QA bands.
    return (
        img.updateMask(mask)
        .select("B[0-9]*")
        .copyProperties(img, ["system:time_start"])
    )

In [ ]:
# apply the QA function to mask poor quality observations
l8_qa = l8.map(qa)

In [ ]:
# apply mean reductions
l8_mean_comp = l8_qa.reduce(ee.Reducer.mean())

# apply median reduction
l8_median_comp = l8_qa.median()

# this is the equivalent calling .reduce() with 
# a median reducer
# image.reduce(ee.Reducer.median)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(l8_median_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Median Composite');
Map.addLayer(l8_mean_comp, {"bands": ["B7_mean","B5_mean","B3_mean"], "min": 0, "max": 5500, "gamma": 1.5}, 'Mean Composite');


Map

### Quality band composite

Sometimes it is useful to create a composite based on a specific variable that is of interest. For example, if we are interested in creating an image of peak vegetation per-pixel we can do that.

In [ ]:
def add_ndvi_band(image):
    ndvi = image.normalizedDifference(['B5','B4']).rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
# apply the ndvi function to each image
l8_ndvi = l8_qa.map(add_ndvi_band)

In [ ]:
# create a composite using the maximum ndvi value
l8_ndvi_comp = l8_ndvi.qualityMosaic("NDVI")
l8_max_comp = l8_ndvi.max()

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(l8_median_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Median Composite');
Map.addLayer(l8_mean_comp, {"bands": ["B7_mean","B5_mean","B3_mean"], "min": 0, "max": 5500, "gamma": 1.5}, 'Mean Composite');
Map.addLayer(l8_ndvi_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'NDVI Composite');
Map.addLayer(l8_max_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Max Composite');


Map

These are examples of compositing techniques. Again, how you composite your imagery will be based on your application. A well-known example of a peer-reviewed compositing approach is the Best Avaiable Pixel (BAP) composite ([White et al., 2014](https://www.tandfonline.com/doi/full/10.1080/07038992.2014.945827)).

It is worth noting that composite images created by reducing an image collection are able to produce pixels in any requested projection and therefore have no fixed output projection. Instead, composites have the [default projection](https://developers.google.com/earth-engine/guides/projections#the-default-projection) of WGS-84 with 1-degree resolution pixels. Composites with the default projection will be computed in whatever output projection is requested. A request occurs by displaying the composite, or by explicitly specifying a projection/scale as in an aggregation such as `reduceRegion()` or `ee.batch.Export`.

## Resampling and Reprojection

Earth Engine makes every effort to handle projection and scale so that you don't have to.  However, there are occasions where an understanding of projections is important to get the output you need.  As an example, it's time to demystify the `reproject()` calls in the previous examples.  Earth Engine requests inputs to your computations in the projection and scale of the output.  The map created using `geemap` has a [Maps Mercator projection](http://epsg.io/3857).  The scale is determined from the map's zoom level.  When you add something to this map, Earth Engine secretly reprojects the input data to Mercator, resampling (with nearest neighbor) to screen resolution pixels based on the map's zoom level, then does all the computations with the reprojected, resampled imagery.  In the previous examples, the `reproject()` calls force the computations to be done at the resolution of the input pixels.


To demonstrate the resampling done by Earth Engine, we are going to re-run the edge detection and display with and without the reprojection.


In [ ]:
# calculate edges without reprojection
ndvi_stddev = ndvi.reduceNeighborhood(
    reducer=ee.Reducer.stdDev(),
    kernel = ee.Kernel.square(4.5)
)

In [ ]:
# force processing at native projection for visualization
ndvi_stddev_reprojected = ndvi_stddev.reproject(proj, None, proj.nominalScale()) 

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 11)

# Display gamma stretches of the input image.
Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(ndvi_stddev, {"min": 0, "max": 1,}, 'NDVI std dev with variable resolution');
Map.addLayer(ndvi_stddev_reprojected, {"min": 0, "max": 1,}, 'NDVI std dev at native resolution');

Map

What's happening here is that the projection specified in `reproject()` (like we used earlier) propagates backwards to the input, forcing all the computations to be performed in that projection.  If you don't specify, the computations are performed in the projection and scale of the map (Mercator) at screen resolution.


You can control how Earth Engine resamples the input with `resample()`.  By default, all resampling is done with nearest neighbor.  To change that, call `resample()` on the inputs.  Compare the input image, resampled to screen resolution with a bilinear and bicubic resampling:

Try zooming in and out, comparing to the input image resampled with nearest neighbor (i.e. without `resample()` called on it).

**_You should rarely, if ever, have to use `reproject()` and `resample()`._** Do not use `reproject()` or `resample()` unless absolutely necessary.  They are only used here for demonstration purposes.